In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, LoadFromDisk, SaveToDisk

import geopandas as gpd

import datetime
import itertools
import time

import ml_rapids

In [2]:
#EOTask for fit and predict
class StreamingModelFit(EOTask):
    """
        Task to fit or predict.
    """
    def __init__(self, time=0, feature_name='bands', label_name='land_cov', model=ml_rapids.HoeffdingTree() ):
        self.time=time
        self.feature_name = feature_name
        self.label_name = label_name
        self.learner = model

    def fit_eopatch(self, eopatch):
        X = eopatch.data[self.feature_name][self.time]
        Y = eopatch.mask_timeless[self.label_name]
        X_train = X.reshape(X.shape[0] * X.shape[1], X.shape[2])
        Y_train = Y.reshape(X.shape[0] * X.shape[1]).astype(int)
        
        self.learner.fit(X_train, Y_train)
        
    def execute(self, eopatch):
        self.fit_eopatch(eopatch)
        return eopatch
            
    def get_model(self):
        return self.learner

In [3]:
#EOTask used for fit
class StreamingModelPredict(EOTask):
    """
        Task to fit or predict.
    """
    def __init__(self, time=0, feature_name='bands', label_name='land_cov', model=ml_rapids.HoeffdingTree() ):
        self.time=time
        self.feature_name = feature_name
        self.label_name = label_name
        self.learner = model
    
    def predict_eopatch(self, eopatch):
        X = eopatch.data[self.feature_name][self.time]
        X_predict = X.reshape(X.shape[0] * X.shape[1], X.shape[2])
        
        Y_predict = self.learner.predict(X_predict)
        Y_predict = Y_predict.reshape(X.shape[0], X.shape[1], 1)
        
        eopatch.add_feature(FeatureType.MASK_TIMELESS, self.feature_name + '_predict', Y_predict)

        return eopatch
    
    def execute(self, eopatch):
        eopatch = self.predict_eopatch(eopatch)
        return eopatch
    
    def get_model(self):
        return self.learner

In [4]:
fit_task = StreamingModelFit()

In [5]:
workflow = LinearWorkflow(
    fit_task
)

In [6]:
#fit
#eopatch prepared in './eopatches/eopatch_test0/'
eopatch = EOPatch.load('./eopatches/eopatch_test0/')
extra_param = {
    fit_task: {'eopatch' : eopatch}
}
workflow.execute(extra_param)

WorkflowResults(
  Dependency(StreamingModelFit):
    EOPatch(
      data: {
        bands: numpy.ndarray(shape=(167, 337, 333, 13), dtype=float32)
      }
      mask: {
        IS_DATA: numpy.ndarray(shape=(167, 337, 333, 1), dtype=bool)
      }
      scalar: {}
      label: {}
      vector: {}
      data_timeless: {}
      mask_timeless: {
        land_cov: numpy.ndarray(shape=(337, 333, 1), dtype=uint32)
      }
      scalar_timeless: {}
      label_timeless: {}
      vector_timeless: {}
      meta_info: {
        maxcc: 0.8
        service_type: 'wcs'
        size_x: '10m'
        size_y: '10m'
        time_difference: datetime.timedelta(days=-1, seconds=86399)
        time_interval: ['2015-07-01', '2018-6-30']
      }
      bbox: BBox(((543473.5912890476, 5139156.267383285), (546805.1886957382, 5142522.075033964)), crs=EPSG:32633)
      timestamp: [datetime.datetime(2015, 7, 11, 10, 0, 8), ..., datetime.datetime(2018, 6, 30, 10, 6, 40)], length=167
    )
)

In [7]:
# get model and use it in future workflows
model = fit_task.get_model()
predict_task = StreamingModelPredict(model=model)

save = SaveToDisk(folder='eopatches', overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

In [8]:
workflow = LinearWorkflow(
    predict_task,
    save
)

In [9]:
#predict
#eopatch prepared in './eopatches/eopatch_test1'
eopatch = EOPatch.load('./eopatches/eopatch_test1/')

extra_param = {
    predict_task: {'eopatch' : eopatch},
    save: {'eopatch_folder': 'hoefding'}
}
workflow.execute(extra_param)

WorkflowResults(
  Dependency(SaveToDisk):
    EOPatch(
      data: {
        bands: numpy.ndarray(shape=(167, 337, 333, 13), dtype=float32)
      }
      mask: {
        IS_DATA: numpy.ndarray(shape=(167, 337, 333, 1), dtype=bool)
      }
      scalar: {}
      label: {}
      vector: {}
      data_timeless: {}
      mask_timeless: {
        bands_predict: numpy.ndarray(shape=(337, 333, 1), dtype=int32)
        land_cov: numpy.ndarray(shape=(337, 333, 1), dtype=uint32)
      }
      scalar_timeless: {}
      label_timeless: {}
      vector_timeless: {}
      meta_info: {
        maxcc: 0.8
        service_type: 'wcs'
        size_x: '10m'
        size_y: '10m'
        time_difference: datetime.timedelta(days=-1, seconds=86399)
        time_interval: ['2015-07-01', '2018-6-30']
      }
      bbox: BBox(((543473.5912890476, 5142522.075033964), (546805.1886957382, 5145887.882684642)), crs=EPSG:32633)
      timestamp: [datetime.datetime(2015, 7, 11, 10, 0, 8), ..., datetime.datetime(2018,